# Darbas su Meta šeimos modeliais

## Įvadas

Šioje pamokoje aptarsime:

- Dviejų pagrindinių Meta šeimos modelių – Llama 3.1 ir Llama 3.2 – apžvalgą
- Kiekvieno modelio naudojimo atvejus ir scenarijus
- Kodo pavyzdį, parodantį kiekvieno modelio išskirtines savybes

## Meta šeimos modeliai

Šioje pamokoje susipažinsime su dviem Meta šeimos arba „Llama bandos“ modeliais – Llama 3.1 ir Llama 3.2

Šie modeliai turi skirtingas versijas ir yra prieinami Github Modelų turgavietėje. Daugiau informacijos apie Github Models naudojimą [AI modelių prototipavimui](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Modelių variantai:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*Pastaba: Llama 3 taip pat galima rasti Github Models, tačiau ši pamoka jos neapims*


## Llama 3.1

Turėdama 405 milijardus parametrų, Llama 3.1 patenka į atvirojo kodo LLM kategoriją.

Šis modelis yra ankstesnės Llama 3 versijos patobulinimas, siūlantis:

- Didesnį konteksto langą – 128 tūkst. žodžių vietoj 8 tūkst.
- Didesnį maksimalų išvesties žodžių skaičių – 4096 vietoj 2048
- Geresnį daugiakalbį palaikymą – dėl padidėjusio mokymo duomenų kiekio

Tai leidžia Llama 3.1 spręsti sudėtingesnes užduotis kuriant GenAI programas, įskaitant:
- Natūralų funkcijų iškvietimą – galimybę kviesti išorinius įrankius ir funkcijas už LLM ribų
- Geresnį RAG našumą – dėl didesnio konteksto lango
- Sintetinių duomenų generavimą – galimybę kurti efektyvius duomenis užduotims, tokioms kaip modelio pritaikymas


### Gimtoji funkcijų iškvietimo galimybė

Llama 3.1 buvo papildomai apmokyta, kad efektyviau naudotų funkcijų ar įrankių iškvietimus. Ji taip pat turi du įmontuotus įrankius, kuriuos modelis gali atpažinti kaip reikalingus naudoti pagal vartotojo užklausą. Šie įrankiai yra:

- **Brave Search** – Gali būti naudojamas gauti naujausią informaciją, pavyzdžiui, orų prognozę, atliekant paiešką internete
- **Wolfram Alpha** – Gali būti naudojamas sudėtingesniems matematiniams skaičiavimams, todėl nereikia rašyti savo funkcijų.

Taip pat galite susikurti savo pasirinktinius įrankius, kuriuos LLM galės iškviesti.

Žemiau pateiktame kodo pavyzdyje:

- Sistemos užklausoje apibrėžiame galimus įrankius (brave_search, wolfram_alpha).
- Išsiunčiame vartotojo užklausą, kuri klausia apie orus tam tikrame mieste.
- LLM atsakys su įrankio iškvietimu į Brave Search įrankį, kuris atrodys taip: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Pastaba: Šiame pavyzdyje įrankis tik iškviečiamas, o jei norite gauti rezultatus, turėsite susikurti nemokamą paskyrą Brave API puslapyje ir apibrėžti pačią funkciją*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Nors Llama 3.1 yra didelis kalbos modelis, viena iš jo ribojimų yra multimodalumas. Tai reiškia, kad jis negali naudoti skirtingų įvesties tipų, pavyzdžiui, vaizdų, kaip užklausų ir pateikti atsakymų. Ši galimybė yra viena pagrindinių Llama 3.2 savybių. Kitos šios versijos naujovės:

- Multimodalumas – gali apdoroti tiek tekstines, tiek vaizdines užklausas
- Mažos ir vidutinės apimties variantai (11B ir 90B) – suteikia lankstumo diegiant modelį,
- Tik tekstiniai variantai (1B ir 3B) – leidžia modelį naudoti kraštiniuose / mobiliuosiuose įrenginiuose ir užtikrina mažą delsą

Multimodalinio palaikymo atsiradimas yra didelis žingsnis atvirojo kodo modelių pasaulyje. Žemiau pateiktame kodo pavyzdyje naudojamas tiek vaizdas, tiek tekstinė užklausa, kad Llama 3.2 90B pateiktų vaizdo analizę.

### Multimodalinio palaikymas su Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Mokymasis čia nesibaigia, tęskite kelionę

Baigę šią pamoką, apsilankykite mūsų [Generatyvaus DI mokymosi kolekcijoje](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), kad toliau gilintumėte savo žinias apie generatyvųjį dirbtinį intelektą!



---

**Atsakomybės atsisakymas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo pradinėje kalboje turėtų būti laikomas autoritetingu šaltiniu. Svarbiai informacijai rekomenduojame profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už bet kokius nesusipratimus ar neteisingą interpretavimą, kilusį naudojantis šiuo vertimu.
